In [1]:
import ollama
from langchain_community.llms.ollama import Ollama

In [2]:
llm = Ollama(model="llama3.2",temperature=0.7)

C:\Users\Hp\AppData\Local\Temp\ipykernel_3868\3885779912.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2",temperature=0.7)


In [4]:
llm.invoke("Hola")

'¡hola! ¿En qué puedo ayudarte hoy?'

In [41]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import ollama


In [6]:
loader = PyPDFLoader("files/brochure.pdf")
paginas = loader.load()

In [7]:
len(paginas)

9

In [8]:
paginas[1].page_content

'Las herramientas basadas sobre Machine \nLearning han tomado fuerza desde hace varios \naños, impulsadas principalmente por el desarrollo \nde la computación, lo cual ha permitido mejorar \nlos tiempos de cómputo y generar cada vez más \ndatos para ser analizados; al mismo tiempo que \nhan mostrado alcanzar mejores resultados en \ncomparación con otras opciones. Por ese \nmotivo, este curso-taller hará una revisión sobre \nqué es Machine Learning, así como cuál es su \nrelación y diferencia con otros conceptos \nsimilares; los cuales tienden a confundirse. \nPosteriormente se revisarán los modelos más \nutilizados en la industria, identificando sus \nventajas y limitaciones, como su correcta forma \nde ser utilizados, reconociendo las situaciones \ndonde podría ser aplicado.\nINFORMACIÓN \nDEL CURSO\n2'

In [9]:
text_splitter = CharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=80,
    separator="\n",
    length_function=len,
)

In [10]:
docs = text_splitter.split_documents(paginas)

In [11]:
len(docs)

23

In [12]:
for i in docs:
    print(i.page_content)
    print("----")

2021
Curso de Capacitación:
MACHINE LEARNING PARA
LA INDUSTRIA
----
Las herramientas basadas sobre Machine 
Learning han tomado fuerza desde hace varios 
años, impulsadas principalmente por el desarrollo 
de la computación, lo cual ha permitido mejorar 
los tiempos de cómputo y generar cada vez más
----
los tiempos de cómputo y generar cada vez más 
datos para ser analizados; al mismo tiempo que 
han mostrado alcanzar mejores resultados en 
comparación con otras opciones. Por ese 
motivo, este curso-taller hará una revisión sobre
----
motivo, este curso-taller hará una revisión sobre 
qué es Machine Learning, así como cuál es su 
relación y diferencia con otros conceptos 
similares; los cuales tienden a confundirse. 
Posteriormente se revisarán los modelos más
----
Posteriormente se revisarán los modelos más 
utilizados en la industria, identificando sus 
ventajas y limitaciones, como su correcta forma 
de ser utilizados, reconociendo las situaciones 
donde podría ser aplicado.
INFORMA

In [13]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma.from_documents(
    docs, embedding=embeddings)

C:\Users\Hp\AppData\Local\Temp\ipykernel_3868\3141230898.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2")


In [47]:
chat = ChatOllama(model="llama3.2",temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil que ayuda a responder preguntas sobre el contenido de un documento."),
    ("human","Pregunta: {question}\n\nContexto: {context}")
])
def ollama_llm(question,context):
    messages = prompt.format_messages(question=question,context=context)
    response = chat.invoke(messages)
    return response.content

In [42]:
llm = Ollama(model="llama3.2",temperature=0.7)
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="Pregunta: {question}\n\nContexto: {context}\n\nRespuesta:"
)
def ollama_llm(question,context):
    formatted_prompt = prompt.format(question=question,context=context)
    response = llm.invoke(formatted_prompt)
    return response

In [33]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
def combine_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
print(type(combine_docs(docs)))

<class 'str'>


In [31]:
# retriever.invoke("¿Qué es LangChain?")
contenido = [i.page_content for i in retriever.invoke("¿Qué vere en el curso?")]


In [37]:
contenido

['como por ejemplo economistas, estadísticos, \ningenieros, administradores o profesionales \nde carreras afines dentro del ámbito de las \ntecnologías de la información y \ncomunicación (TIC), entre otros.\n4\n¿POR QUÉ PUCP?',
 'comunicación (TIC), entre otros.\n4\n¿POR QUÉ PUCP?\nTe ofrecemos formar parte de una gran \ncomunidad académica  (Puesto 15 a nivel \nLatinoamérica en QS World University \nRankings).',
 'Artificial. Investigadora del Grupo de Inteligencia Artificial (IA-PUCP). \nEspecialista en Aprendizaje de máquina, Inteligencia Artificial, \nComputación Evolutiva, Procesamiento de Señales y Minería de datos.']

In [48]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [49]:
rag_chain("¿Cuales son los horarios del curso?")

'Según el documento, los horarios del curso son:\n\n* Sábados de 9:00 am a 12:00 pm.'